## Load libraries

In [ ]:
pip install torch transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import io
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import BertTokenizerFast, BertForTokenClassification
from transformers import pipeline
import torch

Mounted at /content/drive


## Long Didlogue medical-NER

### Load dataset


In [ ]:
# Paths to the files
path_train_long = '/content/drive/MyDrive/W266_Project/Data/train_long.csv'
path_test_long = '/content/drive/MyDrive/W266_Project/Data/test_long.csv'
path_val_long = '/content/drive/MyDrive/W266_Project/Data/val_long.csv'

# Load the datasets
train_long = pd.read_csv(path_train_long)
test_long = pd.read_csv(path_test_long)
val_long = pd.read_csv(path_val_long)

# Check the first few rows to ensure they're loaded correctly
print("Long_dialogue_train_data:")
print(train_long.head())

# Check the shape
print("Shape of Long_dialogue_train/test/val_data:")
print(train_long.shape)
print(test_long.shape)
print(val_long.shape)



Long_dialogue_train_data:
                                            dialogue  \
0  doctor donna torres , date of birth , 08/01/19...   
1  doctor: Good morning, Mr. patient. I'm Dr. doc...   
2  doctor: Hello Mrs. patient, thank you for comi...   
3  doctor hi virginia how're you today patient i'...   
4  doctor: Hello, Mrs. patient, welcome back. How...   

                                                note  Dialogue_Length  \
0  SUBJECTIVE CHIEF COMPLAINT Annual health maint...             8595   
1  SUBJECTIVE CHIEF COMPLAINT Patient reports fru...             1760   
2  SUBJECTIVE CHIEF COMPLAINT Left arm pain after...             4074   
3  SUBJECTIVE CHIEF COMPLAINT Right knee pain. HI...             6728   
4  SUBJECTIVE CHIEF COMPLAINT Recurrent low back ...             1841   

   Note_Length  
0         2794  
1         1536  
2         2971  
3         2207  
4         1595  
Shape of Long_dialogue_train/test/val_data:
(1102, 4)
(180, 4)
(96, 4)


In [ ]:
# find the dialogue with the longest summary to exam the extraction
# find the index of with the max dialogue_length
max_dialogue_train_length_index = train_long['Dialogue_Length'].idxmax()
max_dialogue_test_length_index = test_long['Dialogue_Length'].idxmax()
max_dialogue_val_length_index = val_long['Dialogue_Length'].idxmax()

# Get the row with the maximum note_length
max_dialogue_train_length_row = train_long.loc[max_dialogue_train_length_index]
max_dialogue_test_length_row = test_long.loc[max_dialogue_test_length_index]
max_dialogue_val_length_row = val_long.loc[max_dialogue_val_length_index]

print("max_dialogue_train")
print(max_dialogue_train_length_row)
print("max_dialogue_test")
print(max_dialogue_test_length_row)
print("max_dialogue_val")
print(max_dialogue_val_length_row)


max_dialogue_train
dialogue           doctor next patient is christine hernandez , u...
note               SUBJECTIVE CHIEF COMPLAINT Annual health maint...
Dialogue_Length                                                13924
Note_Length                                                     2545
Name: 548, dtype: object
max_dialogue_test
dialogue           doctor eugene walker , n- date of birth 4/14/1...
note               SUBJECTIVE CHIEF COMPLAINT Annual health maint...
Dialogue_Length                                                 8694
Note_Length                                                     1690
Name: 121, dtype: object
max_dialogue_val
dialogue           doctor sophia brown . date of birth , 3/17/194...
note               SUBJECTIVE CHIEF COMPLAINT Annual health maint...
Dialogue_Length                                                 7384
Note_Length                                                     2280
Name: 7, dtype: object


### medical_NER on long dialogue
- Just run NER on **summary** to identity which entities are most helpful
- model_max_length: 1000000000000000019884624838656, as the max lenggth in summary is 13924, set the max_lenght  to 15000 to save memory and run time



#### medical_NER on long dialogue _ train dataset

In [ ]:
# Step 1: Load tokenizer and model for ClinicalBERT adapted for token classification
#model_name = "medicalai/ClinicalBERT"
model_name_medicalai_ClinicalBERT = "Clinical-AI-Apollo/Medical-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name_medicalai_ClinicalBERT)
model = AutoModelForTokenClassification.from_pretrained(model_name_medicalai_ClinicalBERT)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/736M [00:00<?, ?B/s]

In [ ]:
# Function to extract entities from text
def extract_entities_from_chunks(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=15000)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].tolist())
    labels = [model.config.id2label[prediction.item()] for prediction in predictions[0]]

    entities = []
    for token, label in zip(tokens, labels):
        if label != 'O' and token not in ['[CLS]', '[SEP]']:  # Exclude non-entity labels and special tokens
            entities.append({"token": token, "label": label})
    return entities

# Function to process the dataframe in batches
def process_in_batches(df, batch_size):
    results = []
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch_entities = batch['dialogue'].apply(lambda x: extract_entities_from_chunks(x))
        results.extend(batch_entities)
    return results

# Process the dataframe in batches of size 2
batch_size = 2
train_Long['dialogue_entities'] = process_in_batches(train_Long, batch_size)


In [ ]:
#diaplay medical entity for first 3 row of note_entities
for index, row in train_long.head(3).iterrows():
  print("dialogue", index)
  for entity in row['dialogue_entities']:
    print(entity)

dialogue 0
{'token': '▁fibro', 'label': 'B-DISEASE_DISORDER'}
{'token': 'aden', 'label': 'I-DISEASE_DISORDER'}
{'token': 'omas', 'label': 'I-DISEASE_DISORDER'}
{'token': '▁40', 'label': 'B-AGE'}
{'token': '▁45', 'label': 'I-DURATION'}
{'token': '▁breast', 'label': 'I-FAMILY_HISTORY'}
{'token': '▁cancer', 'label': 'I-DISEASE_DISORDER'}
{'token': '▁cancer', 'label': 'B-DISEASE_DISORDER'}
{'token': '▁breast', 'label': 'B-BIOLOGICAL_STRUCTURE'}
{'token': '▁ultrasound', 'label': 'B-DIAGNOSTIC_PROCEDURE'}
{'token': '▁business', 'label': 'I-DATE'}
{'token': '▁days', 'label': 'I-DATE'}
dialogue 1
{'token': '▁treated', 'label': 'I-HISTORY'}
{'token': '▁Seroquel', 'label': 'B-MEDICATION'}
{'token': '▁anxiety', 'label': 'B-DISEASE_DISORDER'}
{'token': '▁random', 'label': 'B-DETAILED_DESCRIPTION'}
{'token': '▁urine', 'label': 'B-DIAGNOSTIC_PROCEDURE'}
{'token': '▁drug', 'label': 'I-DIAGNOSTIC_PROCEDURE'}
{'token': '▁screening', 'label': 'I-DIAGNOSTIC_PROCEDURE'}
{'token': '▁phone', 'label': 'I-THE

- From the output, we can tell this model use **SentencePiece** tokenlization: token starts with "_" indicates the start of new words or subwords and Tokens without the underscore are continuation tokens that are part of a word or subword unit.
This is different than the tokenization of **Bio-BERT diease** whcih using **wordPiecec**

#### Merge the tokens into words and conbine B and I lable with same entity

In [ ]:
def merge_tokens_and_labels(token_label_pairs):
    merged_results = []
    current_phrase = ""
    current_label = ""

    # Handle the case where token_label_pairs might not be in the expected format
    if not isinstance(token_label_pairs, list) or not all(isinstance(item, dict) for item in token_label_pairs):
        return merged_results # Return empty list if format is unexpected

    for pair in token_label_pairs:
        # Check if keys exist before accessing them
        if 'token' in pair and 'label' in pair:
            token = pair['token']
            label = pair['label']

            # Remove the leading underscore if it exists
            if token.startswith('▁'):
                token = token[1:]

            # If the label starts with B, we start a new phrase
            if label.startswith('B-'):
                # If there's an existing phrase, add it to the results
                if current_phrase:
                    merged_results.append({'phrase': current_phrase, 'label': current_label})
                current_phrase = token
                current_label = label[2:]  # Remove the B- prefix
            elif label.startswith('I-') and label[2:] == current_label:
                # If the label is I- and matches the current entity type, continue the phrase
                current_phrase += " " + token
            else:
                # Handle case where I- doesn't match the current_label, which should be rare
                if current_phrase:
                    merged_results.append({'phrase': current_phrase, 'label': current_label})
                current_phrase = token
                current_label = label[2:]  # Use the new label
        else:
            print(f"Warning: Skipping pair due to missing keys: {pair}") # Alert the user about potential issues in the data

    # Add the last phrase if exists
    if current_phrase:
        merged_results.append({'phrase': current_phrase, 'label': current_label})

    return merged_results


#apply to the Apply batch processing to the dialogue columns of the subset
train_long['dialogue_entities_merged'] = train_long['dialogue_entities'].apply(lambda x: merge_tokens_and_labels(x))

#Check the result
train_long.head(3)

,dialogue,note,Dialogue_Length,Note_Length,dialogue_entities,dialogue_entities_merged
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,8595,2794,"[{'token': '▁fibro', 'label': 'B-DISEASE_DISOR...","[{'phrase': 'fibro aden omas', 'label': 'DISEA..."
1,"doctor: Good morning, Mr. patient. I'm Dr. doc...",SUBJECTIVE CHIEF COMPLAINT Patient reports fru...,1760,1536,"[{'token': '▁treated', 'label': 'I-HISTORY'}, ...","[{'phrase': 'treated', 'label': 'HISTORY'}, {'..."
2,"doctor: Hello Mrs. patient, thank you for comi...",SUBJECTIVE CHIEF COMPLAINT Left arm pain after...,4074,2971,"[{'token': 'lipid', 'label': 'I-DISEASE_DISORD...","[{'phrase': 'lipid', 'label': 'DISEASE_DISORDE..."


### Save extracted NER to new file

In [ ]:
# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_extraction'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Save to CSV
train_long.to_csv(os.path.join(output_dir, 'train_Long_clinical_NER.csv'), index=False)

## Short Dialogue medical NER

### Load the data and experiment on the sub dataset

In [ ]:
# Paths to the files
path_train_short = '/content/drive/MyDrive/W266_Project/Data/train_short.csv'
path_test_short = '/content/drive/MyDrive/W266_Project/Data/test_short.csv'
path_val_short = '/content/drive/MyDrive/W266_Project/Data/val_short.csv'

# Load the datasets
train_short = pd.read_csv(path_train_short)
test_short = pd.read_csv(path_test_short)
val_short = pd.read_csv(path_val_short)

# Check the first few rows to ensure they're loaded correctly
print("Short_dialogue_train_data:")
print(train_short.head())

# Check the shape
print("Shape of Short_dialogue_train_data:")
print(train_short.shape)

# Check the max word length of summary fot furture use
print("Max word length of short dialogue trian:")
print(train_short['Dialogue_Length'].max())
print("Max word length of short dialogue val:")
print(val_short['Dialogue_Length'].max())
print("Max word length of short dialogue test:")
print(test_short['Dialogue_Length'].max())


Short_dialogue_train_data:
  section_header                                       section_text  \
0          GENHX  The patient is a 75-year-old female who comes ...   
1      FAM/SOCHX         Significant for diabetes and hypertension.   
2  PASTMEDICALHX                  Significant for anxiety disorder.   
3          GENHX  The patient is a 77-year-old female who is una...   
4      FAM/SOCHX                                   Noncontributory.   

                                            dialogue  Dialogue_Length  \
0  Doctor: Welcome to the clinic. I am Doctor Fra...             1396   
1  Doctor: Does anyone else in your family suffer...              175   
2  Doctor: Have we gone over your survey results ...              256   
3  Guest_clinician: How old is the patient? Docto...              438   
4  Doctor: Do you have a known- Patient: Drug all...              105   

   Summary_Length  
0             677  
1              42  
2              33  
3             325  
4      

### Medical_NER on short dialogue
- Just run NER on **summary** to identity which entities are most helpful
- model_max_length: 1000000000000000019884624838656, as the max lenggth in summary is 8954, set the max_lenght  to 15000 to save memory and run time

In [ ]:

# Function to extract entities from text
def extract_entities_from_chunks(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=15000)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].tolist())
    labels = [model.config.id2label[prediction.item()] for prediction in predictions[0]]

    entities = []
    for token, label in zip(tokens, labels):
        if label != 'O' and token not in ['[CLS]', '[SEP]']:  # Exclude non-entity labels and special tokens
            entities.append({"token": token, "label": label})
    return entities

# Function to process the dataframe in batches
def process_in_batches(df, batch_size):
    results = []
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch_entities = batch['dialogue'].apply(lambda x: extract_entities_from_chunks(x))
        results.extend(batch_entities)
    return results

# Process the dataframe in batches of size 2
batch_size = 2
train_short['dialogue_entities'] = process_in_batches(train_short, batch_size)

# Display a few rows
train_short.head(3)


,section_header,section_text,dialogue,Dialogue_Length,Summary_Length,dialogue_entities
0,GENHX,The patient is a 75-year-old female who comes ...,Doctor: Welcome to the clinic. I am Doctor Fra...,1396,677,"[{'token': '▁stroke', 'label': 'B-DISEASE_DISO..."
1,FAM/SOCHX,Significant for diabetes and hypertension.,Doctor: Does anyone else in your family suffer...,175,42,"[{'token': '▁family', 'label': 'I-HISTORY'}, {..."
2,PASTMEDICALHX,Significant for anxiety disorder.,Doctor: Have we gone over your survey results ...,256,33,"[{'token': '▁survey', 'label': 'B-DIAGNOSTIC_P..."


In [ ]:
#diaplay medical entity for first 3 row of note_entities
for index, row in train_short.head(3).iterrows():
  print("dialogue", index)
  for entity in row['dialogue_entities']:
    print(entity)

dialogue 0
{'token': '▁stroke', 'label': 'B-DISEASE_DISORDER'}
{'token': '▁This', 'label': 'I-DATE'}
{'token': '▁morning', 'label': 'I-DATE'}
{'token': '▁something', 'label': 'B-SIGN_SYMPTOM'}
{'token': '▁throat', 'label': 'B-BIOLOGICAL_STRUCTURE'}
{'token': '▁dizzy', 'label': 'B-SIGN_SYMPTOM'}
{'token': '▁left', 'label': 'B-BIOLOGICAL_STRUCTURE'}
{'token': '▁hand', 'label': 'I-BIOLOGICAL_STRUCTURE'}
{'token': '▁jaw', 'label': 'B-BIOLOGICAL_STRUCTURE'}
{'token': '▁numb', 'label': 'B-SIGN_SYMPTOM'}
{'token': '▁consciousness', 'label': 'I-SIGN_SYMPTOM'}
{'token': '▁woke', 'label': 'B-CLINICAL_EVENT'}
{'token': '▁up', 'label': 'B-CLINICAL_EVENT'}
{'token': '▁morning', 'label': 'I-DATE'}
{'token': '▁disoriented', 'label': 'B-SIGN_SYMPTOM'}
{'token': '▁it', 'label': 'I-FAMILY_HISTORY'}
{'token': '▁was', 'label': 'I-FAMILY_HISTORY'}
{'token': '▁the', 'label': 'I-FAMILY_HISTORY'}
{'token': '▁middle', 'label': 'I-FAMILY_HISTORY'}
{'token': '▁of', 'label': 'I-DETAILED_DESCRIPTION'}
{'token': '▁

In [ ]:
#apply to the Apply batch processing to the dialogue columns of the subset
train_short['dialogue_entities_merged'] = train_short['dialogue_entities'].apply(lambda x: merge_tokens_and_labels(x))

#Check the result
train_short.head(3)

,section_header,section_text,dialogue,Dialogue_Length,Summary_Length,dialogue_entities,dialogue_entities_merged
0,GENHX,The patient is a 75-year-old female who comes ...,Doctor: Welcome to the clinic. I am Doctor Fra...,1396,677,"[{'token': '▁stroke', 'label': 'B-DISEASE_DISO...","[{'phrase': 'stroke', 'label': 'DISEASE_DISORD..."
1,FAM/SOCHX,Significant for diabetes and hypertension.,Doctor: Does anyone else in your family suffer...,175,42,"[{'token': '▁family', 'label': 'I-HISTORY'}, {...","[{'phrase': 'family', 'label': 'HISTORY'}, {'p..."
2,PASTMEDICALHX,Significant for anxiety disorder.,Doctor: Have we gone over your survey results ...,256,33,"[{'token': '▁survey', 'label': 'B-DIAGNOSTIC_P...","[{'phrase': 'survey', 'label': 'DIAGNOSTIC_PRO..."


### Save extracted NER to new file

In [ ]:
# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/short_dialogue_NER_extraction'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Save to CSV
train_short.to_csv(os.path.join(output_dir, 'train_short_clinical_NER.csv'), index=False)